In [1]:
import pandas as pd

df = pd.read_csv("../data/clean/dataset_base_v21.csv")
df.head()


,campaign,categoria,confianza,confidence,dataset_split,domain,entidad,free_hosting,inclusion,is_https,...,url_norm,domain_complexity,host_entropy,domain_whitelist_score,suspicious_path_token,token_density,trusted_token_context,infra_risk,fake_tld_in_subdomain_or_path,param_count_boost
0,NaN,banca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,https://www.caixabank.es/particular/banca-digi...,0.508495,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0
1,NaN,NaN,NaN,95.0,NaN,NaN,Ibercaja,0.0,NaN,1.0,...,https://www.ibercaja.es/particulares,0.501250,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
2,NaN,NaN,NaN,90.0,NaN,NaN,WiZink,0.0,NaN,1.0,...,https://www.wizink.es/tarjetas,0.451477,0.0,0.0,0.0,0.500,0.0,0.0,0.0,0.0
3,NaN,NaN,NaN,95.0,NaN,NaN,WiZink,0.0,NaN,1.0,...,https://www.wizink.es,0.451477,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0
4,NaN,NaN,NaN,90.0,NaN,NaN,Cetelem,0.0,NaN,1.0,...,https://www.cetelem.es/credito-y-prestamos,0.456514,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0


In [2]:
df["infra_risk"].describe()


count    492.000000
mean       0.614634
std        1.163213
min        0.000000
25%        0.000000
50%        0.000000
75%        0.300000
max        3.300000
Name: infra_risk, dtype: float64

In [3]:
df.groupby("label")["infra_risk"].describe()


,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
0,244.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,248.0,1.219355,1.396188,0.0,0.0,0.3,3.0,3.3


In [6]:
df[(df["label"] == 1) & (df["infra_risk"] == 0)][["url","domain"]].head(20)


,url,domain
244,https://caixabank-es-883f1e.ingress-erytho.eas...,caixabank-es-883f1e.ingress-erytho.easywp.com
248,https://www.mobile.kinman.com/.well-known/acme...,www.mobile.kinman.com
249,https://wififpt.com.vn/es/bankia.es/es/acceso-...,wififpt.com.vn
250,https://web-5.builderallwppro.com/necorreos/vv...,web-5.builderallwppro.com
259,https://personal-santanders-pago-movil-es-app-...,personal-santanders-pago-movil-es-app-subscrip...
260,https://bancosantander-es-online.preview-domai...,bancosantander-es-online.preview-domain.com
263,https://particulares-bancosantander-es.preview...,particulares-bancosantander-es.preview-domain.com
264,https://particulares-bancosantander-es-beyogal...,particulares-bancosantander-es-beyogal85271590...
265,https://particulares-bancosantander.codeanyapp...,particulares-bancosantander.codeanyapp.com
267,https://loikolaka-1.builderallwppro.com/correo...,loikolaka-1.builderallwppro.com


In [7]:
# ------------------------------------------------------
# MEJORA 1: ingress-risk (prueba aislada)
# ------------------------------------------------------

def ingress_risk_boost(url):
    try:
        host = url.split("/")[2].lower()
    except:
        return 0.0
    return 0.5 if "ingress-" in host else 0.0

df["infra_risk_ingress_v1"] = df["infra_risk"] + df["url"].apply(ingress_risk_boost)

print("### Distribución por clase ###")
display(df.groupby("label")["infra_risk_ingress_v1"].describe())

print("\n### FP potenciales (legítimos con valor > 1) ###")
display(df[(df["label"] == 0) & (df["infra_risk_ingress_v1"] > 1)][["url","infra_risk_ingress_v1"]].head(10))

print("\n### FN (phishing con infra_risk_ingress_v1 <= 0) ###")
display(df[(df["label"] == 1) & (df["infra_risk_ingress_v1"] <= 0)][["url","infra_risk","infra_risk_ingress_v1"]].head(10))


### Distribución por clase ###


,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
0,244.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
1,248.0,1.38871,1.606475,0.0,0.0,0.3,3.5,3.8



### FP potenciales (legítimos con valor > 1) ###


,url,infra_risk_ingress_v1



### FN (phishing con infra_risk_ingress_v1 <= 0) ###


,url,infra_risk,infra_risk_ingress_v1
248,https://www.mobile.kinman.com/.well-known/acme...,0.0,0.0
249,https://wififpt.com.vn/es/bankia.es/es/acceso-...,0.0,0.0
250,https://web-5.builderallwppro.com/necorreos/vv...,0.0,0.0
259,https://personal-santanders-pago-movil-es-app-...,0.0,0.0
260,https://bancosantander-es-online.preview-domai...,0.0,0.0
263,https://particulares-bancosantander-es.preview...,0.0,0.0
264,https://particulares-bancosantander-es-beyogal...,0.0,0.0
265,https://particulares-bancosantander.codeanyapp...,0.0,0.0
267,https://loikolaka-1.builderallwppro.com/correo...,0.0,0.0
268,https://informacion-cliente-spainespain3412230...,0.0,0.0


In [9]:
# ------------------------------------------------------
# MEJORA 2: suspicious_hosting_risk (prueba aislada)
# ------------------------------------------------------

SUSPICIOUS_HOSTINGS = [
    "cloudwaysapps.com",
    "wcomhost.com",
    "webspaceguy.com",
    "builderallwppro.com",
    "preview-domain.com",
    "codeanyapp.com",
]

def suspicious_hosting_boost(url):
    try:
        host = url.split("/")[2].lower()
    except:
        return 0.0
    
    for sh in SUSPICIOUS_HOSTINGS:
        if host.endswith(sh):
            return 0.5   # boost de prueba
    return 0.0

df["infra_risk_hosting_v1"] = df["infra_risk"] + df["url"].apply(suspicious_hosting_boost)

print("### Distribución por clase ###")
display(df.groupby("label")["infra_risk_hosting_v1"].describe())

print("\n### FP potenciales (legítimos con valor > 1) ###")
display(df[(df["label"] == 0) & (df["infra_risk_hosting_v1"] > 1)][["url","infra_risk_hosting_v1"]].head(10))

print("\n### FN (phishing con infra_risk_hosting_v1 <= 0) ###")
display(df[(df["label"] == 1) & (df["infra_risk_hosting_v1"] <= 0)][["url","domain","infra_risk","infra_risk_hosting_v1"]].head(15))


### Distribución por clase ###


,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
0,244.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,248.0,1.265726,1.364914,0.0,0.0,0.5,3.0,3.3



### FP potenciales (legítimos con valor > 1) ###


,url,infra_risk_hosting_v1



### FN (phishing con infra_risk_hosting_v1 <= 0) ###


,url,domain,infra_risk,infra_risk_hosting_v1
244,https://caixabank-es-883f1e.ingress-erytho.eas...,caixabank-es-883f1e.ingress-erytho.easywp.com,0.0,0.0
248,https://www.mobile.kinman.com/.well-known/acme...,www.mobile.kinman.com,0.0,0.0
249,https://wififpt.com.vn/es/bankia.es/es/acceso-...,wififpt.com.vn,0.0,0.0
271,https://cuentas.170-64-148-61.cprapid.com/app/...,cuentas.170-64-148-61.cprapid.com,0.0,0.0
273,https://cuentas.15-237-142-160.cprapid.com/bbv...,cuentas.15-237-142-160.cprapid.com,0.0,0.0
276,https://am-coaching.salamander.dev/caixa/vbv/h...,am-coaching.salamander.dev,0.0,0.0
296,https://banco.online.bbva.146-70-81-49.cprapid...,banco.online.bbva.146-70-81-49.cprapid.com,0.0,0.0
297,https://bancosantander.es.hotelparadis.es/avis...,bancosantander.es.hotelparadis.es,0.0,0.0
335,https://correosinfoweb.tempurl.host/infoswebse...,correosinfoweb.tempurl.host,0.0,0.0
336,https://demo2.cloudwp.dev/trial-2tt3605y/corre...,demo2.cloudwp.dev,0.0,0.0


In [11]:
df[(df["label"] == 1) & (df["infra_risk_hosting_v1"] <= 0)][["url","domain","infra_risk","infra_risk_hosting_v1"]].head(20)


,url,domain,infra_risk,infra_risk_hosting_v1
244,https://caixabank-es-883f1e.ingress-erytho.eas...,caixabank-es-883f1e.ingress-erytho.easywp.com,0.0,0.0
248,https://www.mobile.kinman.com/.well-known/acme...,www.mobile.kinman.com,0.0,0.0
249,https://wififpt.com.vn/es/bankia.es/es/acceso-...,wififpt.com.vn,0.0,0.0
271,https://cuentas.170-64-148-61.cprapid.com/app/...,cuentas.170-64-148-61.cprapid.com,0.0,0.0
273,https://cuentas.15-237-142-160.cprapid.com/bbv...,cuentas.15-237-142-160.cprapid.com,0.0,0.0
276,https://am-coaching.salamander.dev/caixa/vbv/h...,am-coaching.salamander.dev,0.0,0.0
296,https://banco.online.bbva.146-70-81-49.cprapid...,banco.online.bbva.146-70-81-49.cprapid.com,0.0,0.0
297,https://bancosantander.es.hotelparadis.es/avis...,bancosantander.es.hotelparadis.es,0.0,0.0
335,https://correosinfoweb.tempurl.host/infoswebse...,correosinfoweb.tempurl.host,0.0,0.0
336,https://demo2.cloudwp.dev/trial-2tt3605y/corre...,demo2.cloudwp.dev,0.0,0.0
